# Lesson 2 Demo 3: Creating Fact and Dimension Tables with Star Schema

<img src="images/postgresSQLlogo.png" width="250" height="250">

## Walk through the basics of modeling data using Fact and Dimension tables.  You will create both Fact and Dimension tables and show how this is a basic element of the Star Schema.

#### Import the library 
Note: An error might popup after this command has exectuted. If it does read it careful before ignoring. 

In [1]:
import psycopg2 as pg

### Create a connection to the database

In [3]:
try:
    conn = pg.connect(host='127.0.0.1', database='postgres', user='postgres', password='alex1988')
except Exception as e:
    print('Error: could not connect to the database')
    print(e)

### Next use that connect to get a cursor that we will use to execute queries.

In [4]:
try:
    cur = conn.cursor()
except Exception as e:
    print('Error could not get cursor')
    print(e)

#### For this demo we will use automactic commit so that each action is commited without having to call conn.commit() after each command. The ability to rollback and commit transactions are a feature of Relational Databases. 

In [5]:
conn.set_session(autocommit=True)

### Let's imagine we work at an online Music Store. There will be many tables in our database but let's just focus on 4 tables around customer purchases. 

`Table Name: customer_transactions
column: Customer Id
column: Store Id
column: Spent`

`Table Name: Customer
column: Customer Id
column: Name
column: Rewards`

`Table Name: store
column: Store Id
column: State`

`Table Name: items_purchased
column: customer id
column: Item Name`

<img src="images/starSchema.png" width="750" height="750">

#### From this representation we can already start to see the makings of a "STAR". We have one fact table (the center of the star) and 3  dimension tables that are coming from it.

### Create the Fact Table and insert the data into the table

In [6]:
try:
    cur.execute("""
                CREATE TABLE IF NOT EXISTS customer_transactions (customer_id int, store_id int, spent int)
                """)
except Exception as e:
    print('Error creating a table')
    print(e)


try:
    query = 'INSERT INTO customer_transactions (customer_id, store_id, spent) VALUES (%s, %s, %s)'
    records = (1,1,20.5)
    cur.execute(query,records)
except Exception as e:
    print('Error inseting data')
    print(e)
    
try:
    query = 'INSERT INTO customer_transactions (customer_id, store_id, spent) VALUES (%s, %s, %s)'
    records = (2,1,35.21)
    cur.execute(query,records)
except Exception as e:
    print('Error inseting data')
    print(e)

### Create our Dimension Tables and insert data into those tables.

In [7]:
# customer
try:
    cur.execute("""
                CREATE TABLE IF NOT EXISTS customer (customer_id int, name varchar, rewards boolean)
                """)
except Exception as e:
    print('Error creating customer talbe')
    print(e)
    
# store
try:
    cur.execute("""
                CREATE TABLE IF NOT EXISTS store (store_id int, state varchar)
                """)
except Exception as e:
    print('Error creating store table')
    print(e)
    
# item_purchased
try:
    cur.execute("""
                CREATE TABLE IF NOT EXISTS items_purchased (customer_id int, item_number int, item_name varchar)
                """)
except Exception as e:
    print('Error creating items_purchased table')
    print(e)
    
# Inserting data

# customer
try:
    query = 'INSERT INTO customer (customer_id, name, rewards) VALUES (%s, %s, %s)'
    record = (1, 'Amanda', True)
    cur.execute(query,record)
except Exception as e:
    print('Error inserting data into customer')
    
try:
    query = 'INSERT INTO customer (customer_id, name, rewards) VALUES (%s, %s, %s)'
    record = (2, 'Toby', False)
    cur.execute(query,record)
except Exception as e:
    print('Error inserting data into customer')
    
# store
try:
    query = 'INSERT INTO store (store_id, state) VALUES (%s, %s)'
    record = (1, 'CA')
    cur.execute(query,record)
except Exception as e:
    print('Error inseting data into store')
    print(e)
    
try:
    query = 'INSERT INTO store (store_id, state) VALUES (%s, %s)'
    record = (2, 'WA')
    cur.execute(query,record)
except Exception as e:
    print('Error inseting data into store')
    print(e)
    
    
# items_purchased
try:
    query = 'INSERT INTO items_purchased (customer_id, item_number, item_name) VALUES (%s, %s, %s)'
    record = (1, 1, 'rubber soul')
    cur.execute(query,record)
except Exception as e:
    print('Error inserting data into items_purchased')
    print(e)
    
try:
    query = 'INSERT INTO items_purchased (customer_id, item_number, item_name) VALUES (%s, %s, %s)'
    record = (2, 3, 'let it be')
    cur.execute(query,record)
except Exception as e:
    print('Error inserting data into items_purchased')
    print(e)

### Now run the following queries on this data that utilize the Fact/Dimension and Star Schema

#### Query 1: Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member. 

#### Query 2: How much did Customer 2 spend

### Query 1:  Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member

In [8]:
try:
    cur.execute("""
                select c.name
                      ,c.rewards
                      ,ct.store_id
                      ,s.state
                      ,ip.item_name 
                      ,ct.spent
                from customer_transactions as ct
                    join customer as c on c.customer_id = ct.customer_id
                    join store as s on s.store_id = ct.store_id
                    join items_purchased as ip on ip.customer_id = ct.customer_id
                where ct.spent > 30
                """)
except Exception as e:
    print('Error querying data')
    print(e)
    

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Toby', 1, 'CA', 'Let It Be', False)


### Query 2: How much did Customer 2 spend?

In [9]:
try:
    cur.execute('select customer_id, spent from customer_transactions where customer_id = 2')
except Exception as e:
    print('Error querying data')
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(2, Decimal('35.21'))


### Summary: You can see here from this elegant schema that we were: 1) able to get "facts/metrics" from our fact table (how much each store sold), and 2) information about our customers that will allow us to do more indepth analytics to get answers to business questions by utilizing our fact and dimension tables. 

### Drop the tables

In [10]:
cur.execute('drop table customer_transactions')
cur.execute('drop table custom')
cur.execute('drop table store')
cur.execute('drop table items_purchased')

### And finally close your cursor and connection. 

In [11]:
cur.close()
conn.close()